In [1]:
import json
import gbench
from gbench import util, report
from gbench.util import *

In [2]:
import plotly.express as px
import pandas as pd
import numpy as np
from functools import reduce
from itertools import accumulate
from IPython.display import display, HTML

In [3]:
keys = ['name', 'real_time', 'cpu_time', 'AShr', 'Add', 'LShr', 'Mul', 'Nodes', 'SDiv', 'SRem', 'Shl', 'Sub', 'UDiv', 'URem', 'pop_count', 'V:AND', 'V:CELLS', 'V:MUX', 'V:NOT', 'V:OR', 'V:XOR']

In [4]:
def get_value(bench, key):
    try:
        value = bench[key]
    except KeyError:
        return np.nan

    try:
        return int(value)
    except ValueError:
        return value

def data_frame(data):
    d = dict()
    for key in keys:
        d[key] = [get_value(bench, key) for bench in data['benchmarks']]
    return pd.DataFrame(data=d)

def load_data(date, benchset):
    path = os.path.join('../results/', date, benchset + '.json')
    with open(path, 'r') as file:
        return data_frame(json.load(file))
    
def load(date):
    return load_data(date, 'noopt'), load_data(date, 'simplify'), load_data(date, 'conjure')

In [5]:
def scatter(lhs, rhs, view):
    lhs_data, lhs_name = lhs
    rhs_data, rhs_name = rhs
    df = pd.merge(lhs_data, rhs_data, how='inner', on='name', 
                  suffixes=["-" + lhs_name, "-" + rhs_name])

    labelx = view + "-" + lhs_name
    labely = view + "-" + rhs_name
    fig = px.scatter(df, x=labelx, y=labely, text='name')

    fig.update_layout(template='plotly_white', hovermode='x')
    
    maximal = max(df[labelx] + df[labely])
    fig.add_shape(
        type='line',
        x0=0,
        y0=0,
        x1=maximal,
        y1=maximal,
        xref='x',
        yref='y',
        line=dict(color='gray', width=1)
    )
    
    fig.update_traces(mode="markers")
    return fig

In [6]:
eqsat = load_data('23-02-16', 'eqsat')
noopt = load_data('23-02-16', 'noopt')
llvm  = load_data('23-02-16', 'llvm')
mux   = load_data('23-02-16', 'mux')
sim_conmul_llvm = load_data('23-02-16', 'sim-conmul-llvm')
sim_conmul_mux_llvm = load_data('23-02-16', 'sim-conmul-mux-llvm')
sim_conmul_mux_llvm_eqsat = load_data('23-02-16', 'sim-conmul-mux-llvm-eqsat')
sim_eqsat_llvm = load_data('23-02-16', 'sim-eqsat-llvm')

In [7]:
sim_conmul_mux_llvm[sim_conmul_mux_llvm['name'].str.startswith("lift_source/\"big")]

,name,real_time,cpu_time,AShr,Add,LShr,Mul,Nodes,SDiv,SRem,...,Sub,UDiv,URem,pop_count,V:AND,V:CELLS,V:MUX,V:NOT,V:OR,V:XOR
48,"lift_source/""big:muls""",76595132457,1349944999,3,177,3,1,1341,0,0,...,0,0,0,0,17163,46354,5880,481,10498,12332
49,"lift_source/""big:divs""",410804561402,619095000,1,66,5,0,1106,1,1,...,0,1,1,0,58072,146420,18526,8994,34218,26610
50,"lift_source/""big:3d_image_toolkit""",3692734790158,5424564000,1,243,89,1,3724,1,1,...,23,1,1,55,54231,141856,17330,6658,33841,29796
51,"lift_source/""big:router_simulator""",3913355845418,5465703999,1,260,89,1,3792,1,1,...,24,1,1,54,68287,176659,21398,9081,41969,35924
52,"lift_source/""big:lan_simulator""",3888931257054,7264965999,3,284,100,1,4332,1,1,...,25,1,1,59,60287,152751,18287,6830,35719,31628
53,"lift_source/""big:tiny86""",1301064129247,92410724000,5,927,150,1,11761,1,1,...,37,1,1,85,161025,359510,34217,12666,79527,72075


In [8]:
def compare(lhs, lhs_suff, rhs, rhs_suff, prefixes):
    df = pd.merge(lhs, rhs, how='inner', on='name', suffixes=["-" + lhs_suff, "-" + rhs_suff])
    filtered = df.loc[:, [col for col in df.columns if any(col.startswith(prefix) for prefix in prefixes)]]
    filtered = filtered.reindex(sorted(filtered.columns), axis=1)
    return pd.concat([df['name'], filtered], axis = 1)

In [9]:
def color_value(val):
    color = 'red' if val > 0 else 'green'
    return f'background-color: {color}'

def make_pretty(styler):
    styler.format(color_value)
    styler.background_gradient(axis=None, vmin=1, vmax=5, cmap="YlGnBu")
    return styler

def difference(frames, keys, relative = False, values = False):
    df = pd.DataFrame()
    for (frame, name) in frames:
        col = frame[keys[0]]
        for key in keys[1:]:
            col += frame[key]
        df[name] = pd.Series(col)
    if relative:
        df.iloc[:] = df.iloc[:].apply(lambda x: (x - df['noopt']) / df['noopt'] * 100)
    elif values:
        df.iloc[:] = df.iloc[:].apply(lambda x: x - df['noopt'])
    return pd.concat([frames[0][0]['name'], df], axis = 1)

In [10]:
relative = False

df = difference([(noopt, "noopt"), (eqsat, "eqsat"), (llvm, "llvm"), (mux, "mux"), (sim_eqsat_llvm, "eqsat-llvm"), (sim_conmul_llvm, "mul-llvm"), (sim_conmul_mux_llvm, "mul-mux-llvm"), (sim_conmul_mux_llvm_eqsat, "mul-mux-llvm-eqsat")], ['V:CELLS'], relative)
view = df[df['name'].str.startswith("lift_source")]

def highlight_max(s, props=''):
    return np.where(s == np.nanmin(s.values), props, '')

columns = ['eqsat', 'llvm', 'mux', 'eqsat-llvm', 'mul-llvm', 'mul-mux-llvm', 'mul-mux-llvm-eqsat']

view.style \
    .apply(lambda x: ["color:red;" if v > x.iloc[0] else "" for v in x], axis = 1, subset=columns) \
    .apply(lambda x: ["color:green;" if v < x.iloc[0] else "" for v in x], axis = 1, subset=columns) \
    .apply(highlight_max, props='color:white;background-color:green', axis=1, subset=columns)

,name,noopt,eqsat,llvm,mux,eqsat-llvm,mul-llvm,mul-mux-llvm,mul-mux-llvm-eqsat
28,"lift_source/""min:small""",2481,2481,2481,2481,2481,2486,2486,2486
29,"lift_source/""min:adcs""",50302,50298,50248,50302,50353,51889,51939,51892
30,"lift_source/""min:adds""",51753,51809,51810,51804,51805,53575,53568,53521
31,"lift_source/""min:ands""",51573,51571,51575,51580,51580,53281,53331,53323
32,"lift_source/""min:calls""",10964,10966,10966,10963,10965,11083,11084,11085
33,"lift_source/""min:cmps""",34307,34306,34303,34316,34310,35379,35377,35378
34,"lift_source/""min:decs""",34313,34311,34313,34316,34311,35376,35370,35374
35,"lift_source/""min:mul-16""",10411,10420,10410,10410,10411,11076,10836,10827
36,"lift_source/""min:idivs""",84396,84395,84396,84392,84396,77766,76677,77181
37,"lift_source/""min:incs""",21163,21162,21161,21163,21163,21535,21536,21536


In [14]:
eqsat = load_data('23-02-17', 'eqsat')
noopt = load_data('23-02-17', 'noopt')
llvm  = load_data('23-02-17', 'llvm')
conall  = load_data('23-02-17', 'conall-llvm')
condiv  = load_data('23-02-17', 'condiv-llvm')
conmul  = load_data('23-02-17', 'conmul-llvm')
conadd = load_data('23-02-17', 'conadd-llvm')
sel  = load_data('23-02-17', 'sim-eqsat-llvm')
scmml = load_data('23-02-17', 'sim-conmul-mux-llvm')
scaml = load_data('23-02-17', 'sim-conall-mux-llvm')
scalmle  = load_data('23-02-17', 'sim-conall-mux-llvm-eqsat')
scmcdml  = load_data('23-02-17', 'sim-conmul-condiv-mux-llvm')
scal  = load_data('23-02-17', 'sim-conall-llvm')

In [19]:
relative = False
values = False

instances = [
    (noopt, "noopt"),
    #(noopt_new, "noopt-new"), 
    (eqsat, "eqsat"), 
    #(llvm, "llvm"), 
    #(sel, "eqsat-ll"), 
    (conadd, "conadd"),
    (conmul, "conmul"),
    (condiv, "condiv"), 
    (conall, "conall"), 
    #(scal, "sim-conall"),
    #(scmml, "conmul-mux"),
    (scaml, "conall-mux"),
    (scmcdml, "conmul-condiv-mux"),
    (scalmle, "conall-mux-eqsat")
]

df = difference(instances, ['V:CELLS'], relative, values)
view = df[df['name'].str.startswith("lift_source")]

columns = [
    'noopt', 
    #'noopt-new',
    'eqsat', 
    #'llvm', 
    #'eqsat-ll',
    'conadd',
    'conmul', 
    'condiv', 
    'conall', 
    #'sim-conall', 
    #'conmul-mux', 
    'conall-mux',
    'conmul-condiv-mux',
    'conall-mux-eqsat'
]

view.style \
    .apply(lambda x: ["color:red;" if v > x.iloc[0] else "" for v in x], axis = 1, subset=columns) \
    .apply(lambda x: ["color:green;" if v < x.iloc[0] else "" for v in x], axis = 1, subset=columns) \
    .apply(highlight_max, props='color:white;background-color:green', axis=1, subset=columns)

,name,noopt,eqsat,conadd,conmul,condiv,conall,conall-mux,conmul-condiv-mux,conall-mux-eqsat
28,"lift_source/""min:small""",2548,2548,2640,2565,2565,2640,2565,2565,2565.000000
29,"lift_source/""min:adcs""",22930,22931,55342,26273,26272,56230,26373,26270,24485.000000
30,"lift_source/""min:adds""",24595,24701,50890,28720,28702,52543,25911,28708,24768.000000
31,"lift_source/""laky:half-ands""",14591,14591,18354,15840,15890,18302,11994,15840,12184.000000
32,"lift_source/""min:ands""",22433,22432,39080,26283,26268,39238,19461,28660,19447.000000
33,"lift_source/""min:calls""",6996,6996,12016,7162,7158,12068,6551,7161,6790.000000
34,"lift_source/""min:cmps""",17457,17468,39987,19556,19563,40205,17859,19563,16608.000000
35,"lift_source/""min:decs""",12679,12679,16788,13396,13396,16212,11909,13396,12060.000000
36,"lift_source/""min:mul-16""",7802,7802,8394,8474,8099,8769,7513,8225,7524.000000
37,"lift_source/""min:idivs""",80367,80367,80607,80586,73770,73792,71213,73144,71044.000000


In [13]:
# new_noopt, new_simpl, new_conju = load('23-02-07')
# abc_noopt, abc_simpl, abc_conju = load('23-01-25')

In [245]:
# pd.merge(new_noopt, abc_noopt, how='inner', on='name')

In [176]:
scatter((noopt, "new-noopt"), (sim_conmul_mux_llvm_eqsat, "sim-conmul-mux-llvm-eqsat"), "V:CELLS")

In [299]:
scatter((new_noopt, "new-noopt"), (abc_noopt, "abc-noopt"), "V:AND")

In [248]:
scatter((new_noopt, "new-noopt"), (new_conju, "new-alu-synthesis"), "V:CELLS")

In [269]:
scatter((new_noopt, "new-noopt"), (new_conju, "new-alu-synthesis"), "Nodes")

In [292]:
scatter((new_noopt, "new-noopt"), (new_conju, "new-alu-synthesis"), "Mul")

In [166]:
noopt

,name,real_time,cpu_time,AShr,Add,LShr,Mul,Nodes,SDiv,SRem,...,Sub,UDiv,URem,pop_count,V:AND,V:CELLS,V:MUX,V:NOT,V:OR,V:XOR
0,"lift_bytes/""min:AND_AL_IMMb""",2972073580,195898000,0,2,0,0,110,0,0,...,0,0,0,1,8089,1393,0,40,731,574
1,"lift_bytes/""min:LEA_GPRv_AGEN_32""",3487134498,189459000,0,5,0,0,155,0,0,...,0,0,0,0,9342,2628,736,61,827,759
2,"lift_bytes/""min:LODSB""",2668726298,189224249,0,2,0,0,110,0,0,...,0,0,0,1,8089,1393,0,40,731,574
3,"lift_bytes/""min:LODSB""",2544468480,180706999,0,4,0,0,107,0,0,...,0,0,0,0,9902,1934,0,41,878,771
4,"lift_bytes/""min:LOOP_RELBRb""",3000810256,185864000,0,7,0,0,111,0,0,...,0,0,0,0,9169,1799,32,67,817,701
5,"lift_bytes/""min:MOVSB""",3295159685,186351000,0,6,0,0,107,0,0,...,0,0,0,0,12672,2629,0,49,1110,1008
6,"lift_bytes/""min:MOV_GPR8_GPR8_8A""",3089338554,195559000,0,2,1,0,156,0,0,...,0,0,0,0,8293,1973,528,57,751,605
7,"lift_bytes/""min:OR_GPRv_IMMz_32""",3321818430,204426999,0,2,0,0,142,0,0,...,0,0,0,1,9377,2133,448,86,844,662
8,"lift_bytes/""min:POPAD_32""",5349704657,196898999,0,15,0,0,127,0,0,...,0,0,0,0,16382,3163,0,95,1467,1356
9,"lift_bytes/""min:PUSH_IMMz_32""",2649305024,188547000,0,4,0,0,102,0,0,...,0,0,0,0,10028,1993,0,71,892,814


In [167]:
df = compare(noopt, "new", sim_conmul_mux_llvm_eqsat, "opt", ["V:CELLS"])
df[df['name'].str.startswith("lift_source")]

,name,V:CELLS-new,V:CELLS-opt
30,"lift_source/""min:small""",2481,2486
31,"lift_source/""min:adcs""",50302,51889
32,"lift_source/""min:adds""",51753,53575
33,"lift_source/""min:ands""",51573,53281
34,"lift_source/""min:calls""",10964,11083
35,"lift_source/""min:cmps""",34307,35379
36,"lift_source/""min:decs""",34313,35376
37,"lift_source/""min:mul-16""",10411,11076
38,"lift_source/""min:idivs""",84396,77766
39,"lift_source/""min:incs""",21163,21535
